In [1]:
import torch.nn as nn
import torch
import torchvision.transforms.functional as functional


In [2]:
# Each unet block has two conv layers

class DoubleConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(DoubleConv, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 3, 1, 1, bias=False), #kernal_size, stride, padding
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),

            nn.Conv2d(out_channels, out_channels, 3, 1, 1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )
    def forward(self, x):
        return self.conv(x)

In [3]:
class UNET2D(nn.Module):
    def __init__(self, in_channels=3, out_channels=1, features = [64, 128, 256, 512]):
        super(UNET2D, self).__init__()
        self.downsamplings = nn.ModuleList()
        self.upsamplings = nn.ModuleList()
        self.pool = nn.MaxPool2d(2, 2)

        #down sampling
        for feature in features:
            self.downsamplings.append(DoubleConv(in_channels, feature))
            in_channels = feature

        #up sampling
        for feature in reversed(features):
            self.upsamplings.append(nn.Sequential(
                    nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True),
                    DoubleConv(feature * 2, feature)  # This ensures the channel count is correct after concatenation
                )
            )
            self.upsamplings.append(DoubleConv(feature*2, feature))

        self.bottleneck = DoubleConv(features[-1], features[-1]*2)
        self.final_conv = nn.Conv2d(features[0], out_channels, kernel_size=1)

    def forward(self, x):
        #skip connection
        skip_connection = []

        for down_sampling in self.downsamplings:
            x = down_sampling(x)
            skip_connection.append(x)
            x = self.pool(x)

        x = self.bottleneck(x)
        skip_connection = skip_connection[::-1] #reverse

        for idx  in range(0, len(self.upsamplings), 2):
            x = self.upsamplings[idx](x)
            skip_connection_feature = skip_connection[idx//2]
            concatenated_features = torch.cat([x, skip_connection_feature], dim=1)
            x = self.upsamplings[idx+1](concatenated_features)


        return self.final_conv(x)

In [5]:
ex = torch.rand((3, 3, 160, 160))
model = UNET2D(in_channels=3, out_channels=1)
out = model(ex)
print(out.shape)

torch.Size([3, 1, 160, 160])
